## 第 0 步：潜在狄利克雷分布 ##

LDA 用于将文档中的文本分类为特定话题。它会用狄利克雷分布构建一个话题/文档模型和单词/话题模型。

* 每个文档都建模为话题多态分布，每个话题建模为单词多态分布。
* LDA 假设我们传入其中的每段文本都相互关联。因此，选择正确的语料库很关键。
* 它还假设文档是根据多种话题创建的。然后，这些话题根据单词的分布概率生成单词。

## 第 1 步：加载数据集

我们将使用的数据集是一个列表，其中包含在 15 年内发表的超过 100 万条新闻标题。首先，我们将从 `abcnews-date-text.csv` 文件中加载该数据集。

In [ ]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

我们来看看该数据集：

In [ ]:
'''
Get the total number of documents
'''
print(len(documents))

In [ ]:
documents[:5]

## 第 2 步：预处理数据 ##

我们将执行以下步骤：

* **标记化**：将文本拆分为句子，将句子拆分为单词。使单词全小写并删除标点。
* 删除少于 3 个字符的单词。
* 删除所有**停止词**。
* **词形还原**单词 - 第三人称的单词变成第一人称，过去式和将来式变成现在式。
* **词干提取**单词 - 将单词简化成根形式。

In [ ]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
import nltk
nltk.download('wordnet')

### Lemmatizer 示例
在预处理数据集之前，我们先看一个词形还原示例。如果词形还原单词“went”，输出是什么：

In [ ]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

### Stemmer 示例
再看一个词干提取示例。我们向 stemmer 中传入多个单词，看看它是如何处理每个单词的：

In [ ]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            
    return result



In [ ]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

In [ ]:
documents

现在预处理所有新闻标题。为此，我们使用 pandas 中的 [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) 函数向 `headline_text` 列应用 `preprocess()`。

**注意**：可能需要几分钟（我的笔记本需要 6 分钟）

In [ ]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = 

In [ ]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

## 第 3.1 步：数据集上的词袋

现在，根据 processed_docs 创建一个字典，后者包含单词在训练集中的出现次数。为此，将 `processed_docs` 传入 [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) 并称之为 `dictionary`。

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

滤除出现在以下情形中的标记

* 出现在 no_below 个以下的文档中（绝对数字），或
* 出现在 no_above 个以上的文档中（ 总语料库大小的一部分，不是绝对数字）。
* 在 (1) 和 (2) 之后，仅保留前 keep_n 个最常见的标记（如果为 None，则保留所有标记）。

In [ ]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above


** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* 将文档（单词列表）转换为词袋格式 = 2 元组（token_id、token_count）列表。每个单词都应该是标记化和标准化的字符串（unicode 或 utf8-编码）。文档中的单词没有进一步预处理了；在调用此函数之前，请应用标记化、词干提取等方法。

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
# TODO
bow_corpus = 

In [ ]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

## 第 3.2 步： 对文档集合应用 TF-IDF ##

虽然使用 gensim 模型的 LDA 实现并非必须对语料库执行 TF-IDF，但是建议这么做。TF-IDF 在初始化过程中要求词袋（整数值）训练语料库。在转换过程中，它将接受向量并返回另一个维度相同的向量。

*请注意：Gensim 的作者规定 LDA 的标准流程是使用词袋模型。*

** TF-IDF 是“词频、逆文本频率"的简称。**

* 它是根据单词在多个文档中的出现频率对单词（或“术语”）重要性进行评分的方式。
* 如果单词频繁出现在文档中，则很重要，给该单词评很高的得分。但是如果单词出现在很多文档中，则不是唯一标识符，给该单词评很低的得分。
* 因此，“the”和“for”等常见单词出现在很多文档中，评分将降低。经常出现在单个文档中的单词评分将升高。

换句话说：

* TF(w) = `（术语 w 出现在文档中的次数）/（文档中的术语总数）`。
* IDF(w) = `log_e（文档总数/包含术语 w 的文档数）`。

** 例如 **

* 假设有一个文档包含 `100` 个单词，其中单词“tiger”出现了 3 次。
* "tiger"的词频（即 tf）是：
    - `TF = (3 / 100) = 0.03`. 

* 现在，假设有 `10 million` 个文档，单词”tiger“出现在了其中 `1000` 个文档中。逆文档频率（即 idk）的计算方式为：
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* 因此，Tf-idf 权重是这些数量的积：
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [ ]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

# TODO
tfidf = 

In [ ]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = 

In [ ]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

## 第 4.1 步：使用词袋运行 LDA ##

我们将处理文档语料库中的 10 个话题。

** 我们将使用所有 CPU 核心运行 LDA，以并行化并加快模型训练。**

我们将调整以下参数：

* **num_topics** 是请求从训练语料库中提取的潜在话题数。
* **id2word** 是从单词 ID（整数）到单词（字符串）的映射，用于判断词汇表大小，以及用于调试和输出话题。
* **workers** 是用于并行化的额外进程数。默认使用所有可用的核心。
* **alpha** 和 **eta** 是影响文档-话题 (θ) 和话题-单词 (lambda) 分布的超参数。暂时使用默认值（默认值为 `1/num_topics`）
    - Alpha 是文档-话题分布。
        * alpha 很高：每个文档都包含所有话题（文档似乎都相似）。
        * alpha 很低：每个文档包含的话题很少

In [ ]:
- Eta 是话题-单词分布。
    * eta 很高：每个话题都包含大部分单词（话题似乎都相似）。
    * eta 很低：每个话题包含的单词很少。

* ** 通过次数** 是通过语料库的训练次数。例如，如果训练语料库有 50,000 个文档，块大小是 10,000，通过次数是 2，则在线训练需要更新 10 次：
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999`

In [ ]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model = 

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

### 主题分类 ###

根据每个话题中的单词及其对应的权重，你能够推断出哪些类别？

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## 第 4.2 步：使用 TF-IDF 运行 LDA ##

In [ ]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
# TODO
lda_model_tfidf = 

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

### 主题分类 ###

可以看出，在使用 tf-idf 时，不太常见的单词权重更高，导致名词被考虑在内。这样就更难分类，因为名词比较难分类。进一步表明我们应用的模型取决于要处理的文本语料库的类型。

根据每个话题中的单词及其对应的权重，你能够推断出哪些类别？

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## 第 5.1 步：通过使用 LDA 词袋模型分类样本文档评估性能##

我们将检查可以在何处分类测试文档。

In [ ]:
'''
Text of sample document 4310
'''
processed_docs[4310]

In [ ]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO
# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

### 它成为我们所分配话题（X，分类正确）的一部分的概率最高 ###

## 第 5.2 步：通过使用 LDA TF-IDF 模型分类样本文档评估性能##

In [ ]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

### 它成为我们所分配话题 (X) 的一部分的概率最高 (`x%`) ###

## 第 6 步：用未见过的文档测试模型 ##

In [ ]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

模型正确地将未见过的文档分类成 X 类别，概率是 x%。